 Do the same thing, but this time assuming that the retrieving agent does
 not know the parameters of the world generating model, just the fact that
 the rate of drift is sampled from a bimodal distribution. Hint: you'll
 have to have the agent learn the drift distribution parameters. EM
 is your friend here. 

In [1]:
# Importing Required libraries
import numpy as np
from drawFromADist import drawFromADist
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture


In [2]:
def get_gaussian_mixture(mean0=0, var0=0.2, mean1=1, var1=0.2, num_of_samples=20000, plot=False):
    y = np.random.normal(mean0, var0, num_of_samples)
    y = np.concatenate((y, np.random.normal(mean1, var1, num_of_samples)))
    if plot:
        plt.hist(y, density=True, bins=50)
        plt.show()
    return y

Using sklearn library GaussianMixture, which will use history delta values and try to cluster them and find the two means and variance, using these predictions, gaussian mixture will sample more deltas while retrieval

In [3]:
def model_3(item_occurance, random_world_m=True, beta_param=0.99):
    N_WORLD_FEATURES = 5
    N_ITEMS = 10
    ENCODING_TIME = 500
    TEST_TIME = 25
    
    schedule_list = np.sort(np.ceil(item_occurance)).reshape(-1, 1)
    item_list = np.array(range(N_ITEMS)).reshape(-1,1)
    schedule = np.concatenate((schedule_list, item_list), axis=1)  # https://numpy.org/doc/stable/reference/generated/numpy.concatenate.html
    schedule_load = ENCODING_TIME/np.median(np.diff(schedule[:,0]))
    encoding = np.zeros((N_ITEMS,N_WORLD_FEATURES+1))
    gaussian_mixture = get_gaussian_mixture(plot=False)
    
    if random_world_m:
        # Generate random world 
        world_m = np.random.randint(low=1, high=5, size=N_WORLD_FEATURES)
    else:
        world_m = np.array([1, 2, 1, 2, 3]) 
    world_var = 1
    #delta = 0.05             
    #beta_param = 0.9  #context drift parameter, ct=beta*ct + c_t-1      # beta_param is in function call
    m = 0
    # simulating encoding
    training_data_delta = []
    for time in range(1,ENCODING_TIME + 1):
        delta=np.random.choice(gaussian_mixture) # CT_In
        training_data_delta.append(delta)
        world_m = beta_param*world_m + delta
        world = np.random.normal(world_m, world_var)

        if(m<N_ITEMS):
            if(time==schedule[m,0]):
                encoding[m] =  np.append(world, m)
                m += 1

    out=np.zeros(TEST_TIME)

    EM_model = GaussianMixture(n_components=2)
    training_data_delta = np.array(training_data_delta).reshape(-1, 1)
    EM_model.fit(training_data_delta)
    
    while time < ENCODING_TIME + TEST_TIME:
        delta=EM_model.sample()[0][0]
        world_m = beta_param*world_m + delta

        world = np.random.normal(world_m, world_var)
        
        soa=np.zeros(N_ITEMS)
        current_world=np.zeros(N_WORLD_FEATURES+1)
        for m in range(N_ITEMS):
            current_world=  np.append(world, m)
            
            soa[m]=np.dot(encoding[m],current_world.T)     
        #NORMALIZE
        soa=soa/np.sqrt(np.sum(soa**2)) 
        # REF: https://stackoverflow.com/questions/21030391/how-to-normalize-a-numpy-array-to-a-unit-vector
        t2 = drawFromADist(soa)
        temp = np.nonzero(t2)[0][0]
        out[time - ENCODING_TIME] = temp # Get the non zero element 
        # REF: https://www.geeksforgeeks.org/numpy-nonzero-in-python/

        time = time + 1
    success=len(np.unique(out))
    return success, schedule_load, EM_model.means_, EM_model.weights_

In [4]:
out = []
item_occurance = [1, 100, 199, 298, 397, 496, 497, 498, 499, 500]
schedule_load = 0
num_of_trials=50
for i in range(num_of_trials):

    success, schedule_load, pred_means, pred_vars = model_3(item_occurance, random_world_m=False)
    out.append(success)
    if i % 10 == 0:
        print(f"For {i}th trial, means predicted by EM Model is", pred_means,"And variance predicted is", pred_vars)
#plt.plot(out)
print(f"Mean of number of items retrieved over {num_of_trials} trial is :{np.mean(out)}. Schedule Load for this schedule:{schedule_load}")


For 0th trial, means predicted by EM Model is [[0.98110302]
 [0.00818178]] And variance predicted is [0.49925095 0.50074905]
For 10th trial, means predicted by EM Model is [[1.00566155]
 [0.00626373]] And variance predicted is [0.48250166 0.51749834]
For 20th trial, means predicted by EM Model is [[ 1.00586041]
 [-0.00427939]] And variance predicted is [0.50060553 0.49939447]
For 30th trial, means predicted by EM Model is [[0.00806899]
 [0.99771203]] And variance predicted is [0.50544725 0.49455275]
For 40th trial, means predicted by EM Model is [[-0.00417216]
 [ 0.98298625]] And variance predicted is [0.49526044 0.50473956]
Mean of number of items retrieved over 50 trial is :8.54. Schedule Load for this schedule:5.05050505050505
